In [22]:
# file: backend/forecast_service.py
import os
from pathlib import Path
from dataclasses import dataclass
from typing import List, Dict, Any, Optional, Tuple

import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
import joblib
import json
import time

In [3]:
# Optional SHAP import check
try:
    import shap
    HAS_SHAP = True
except Exception:
    HAS_SHAP = False


In [12]:
# -------------------------
# 1) Data loader: M5 -> long panel
# -------------------------
def load_m5_long(m5_dir: str, store_ids: Optional[List[str]] = None, sku_ids: Optional[List[str]] = None,
                 max_items: int = 1000, use_validation: bool = True) -> pd.DataFrame:
    """
    Load M5 sales -> long format and merge calendar + prices.
    Returns DataFrame with columns: date, id, item_id, dept_id, cat_id, store_id, state_id, demand, price, calendar...
    """
    m5_dir = Path(m5_dir)
    sales_file = "sales_train_validation.csv" if use_validation else "sales_train_evaluation.csv"
    sales = pd.read_csv(m5_dir / sales_file)
    cal = pd.read_csv(m5_dir / "calendar.csv")
    prices = pd.read_csv(m5_dir / "sell_prices.csv")

    # Optionally filter stores or specific SKUs to limit size
    if store_ids is not None:
        sales = sales[sales["store_id"].isin(store_ids)].copy()

    if sku_ids is not None:
        sales = sales[sales["id"].isin(sku_ids)].copy()
    else:
        sales = sales.head(max_items).copy()

    d_cols = [c for c in sales.columns if c.startswith("d_")]
    id_cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]

    long = sales[id_cols + d_cols].melt(
        id_vars=id_cols,
        value_vars=d_cols,
        var_name="d",
        value_name="demand",
    )

    # merge calendar (d -> date + events etc.)
    cal["date"] = pd.to_datetime(cal["date"])
    long = long.merge(cal, on="d", how="left")

    # merge price using (store_id, item_id, wm_yr_wk)
    long = long.merge(prices, on=["store_id", "item_id", "wm_yr_wk"], how="left")
    long = long.rename(columns={"sell_price": "price"})
    long["demand"] = long["demand"].astype(float)
    long = long.sort_values(["id", "date"]).reset_index(drop=True)
    return long


In [5]:
# -------------------------
# 2) Feature engineering per-SKU panel
# -------------------------
def add_panel_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Adds lag features, rolling stats, calendar encodings, SNAP, price-driven promo flag.
    Input: df with columns ['id','date','demand','price', calendar cols...]
    """
    df = df.copy()
    df = df.sort_values(["id", "date"])

    # date parts
    df["dow"] = df["date"].dt.dayofweek
    df["month"] = df["date"].dt.month
    df["year"] = df["date"].dt.year
    # wday exists in calendar.csv as numeric day-of-week starting at 1, but use dow for model

    # event flags
    df["has_event_1"] = df["event_name_1"].notna().astype(int)
    df["has_event_2"] = df["event_name_2"].notna().astype(int)
    # snap flags
    for c in ["snap_CA", "snap_TX", "snap_WI"]:
        if c in df.columns:
            df[c] = df[c].fillna(0).astype(int)

    # price features
    df["price"] = df["price"].astype(float)
    df["price_lag_1"] = df.groupby("id")["price"].shift(1)
    df["price_change_1"] = (df["price"] / df["price_lag_1"] - 1).replace([np.inf, -np.inf], np.nan)
    # promo flag: price below rolling median of last 28 days (exclude today)
    df["price_rolling_med_28"] = df.groupby("id")["price"].shift(1).rolling(28).median().reset_index(level=0, drop=True)
    df["promo_flag"] = ((df["price"] < df["price_rolling_med_28"]).astype(float)).fillna(0.0)

    # demand lags & rolling stats
    for lag in [1, 7, 14, 28]:
        df[f"lag_{lag}"] = df.groupby("id")["demand"].shift(lag)

    df["roll_mean_7"] = df.groupby("id")["demand"].shift(1).rolling(7).mean().reset_index(level=0, drop=True)
    df["roll_std_7"]  = df.groupby("id")["demand"].shift(1).rolling(7).std().reset_index(level=0, drop=True)
    df["roll_mean_28"]= df.groupby("id")["demand"].shift(1).rolling(28).mean().reset_index(level=0, drop=True)
    df["roll_std_28"] = df.groupby("id")["demand"].shift(1).rolling(28).std().reset_index(level=0, drop=True)

    # fill some small NaNs created by price features
    df["price_change_1"] = df["price_change_1"].fillna(0.0)
    df["promo_flag"] = df["promo_flag"].fillna(0.0)

    return df


In [30]:
# -------------------------
# 3) Training service (global model)
# -------------------------
@dataclass
class GlobalForecastModel:
    model_mean: Any = None
    model_q50: Any = None
    model_q90: Any = None
    feature_cols: List[str] = None
    cat_cols: List[str] = None
    last_train_date: Optional[pd.Timestamp] = None
    train_metrics: Dict[str, Any] = None

    def train(self, df_long: pd.DataFrame, val_days: int = 28, lgb_params: Optional[Dict] = None):
        """
        Train global LightGBM mean + quantile models.
        df_long: output of load_m5_long()
        """
        t0 = time.time()
        df = df_long.copy()
        df["date"] = pd.to_datetime(df["date"])
        df = add_panel_features(df)

        # choose feature list (drop ones that don't exist)
        cat_cols = [c for c in ["id","item_id","dept_id","cat_id","store_id","state_id"] if c in df.columns]
        base_cols = [
            "dow","month","year",
            "has_event_1","has_event_2",
            "snap_CA","snap_TX","snap_WI",
            "price","price_change_1","promo_flag",
            "lag_1","lag_7","lag_14","lag_28",
            "roll_mean_7","roll_std_7","roll_mean_28","roll_std_28"
        ]
        base_cols = [c for c in base_cols if c in df.columns]

        # drop rows with missing features (necessary for initial training)
        df_model = df.dropna(subset=base_cols).copy()

        # time split (global): last val_days days as validation
        global_max_date = df_model["date"].max()
        split_date = global_max_date - pd.Timedelta(days=val_days)
        train = df_model[df_model["date"] <= split_date]
        val   = df_model[df_model["date"] >  split_date]

        X_train = train[cat_cols + base_cols]
        y_train = train["demand"].values
        X_val = val[cat_cols + base_cols]
        y_val = val["demand"].values

        # ensure categorical dtype
        for c in cat_cols:
            X_train[c] = X_train[c].astype("category")
            X_val[c] = X_val[c].astype("category")

        # default LGB params (you can tune)
        if lgb_params is None:
            lgb_params = dict(
                n_estimators=2000,
                learning_rate=0.03,
                num_leaves=128,
                subsample=0.8,
                colsample_bytree=0.8,
                random_state=42,
                verbose=-1
            )

        # mean model
        self.model_mean = LGBMRegressor(**lgb_params)
        self.model_mean.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            callbacks=[
                early_stopping(stopping_rounds=100),
                log_evaluation(period=100),  # <-- replaces verbose=100
            ],
        )

        val_pred = self.model_mean.predict(X_val)
        mae = mean_absolute_error(y_val, val_pred)

        # quantile models
        params_q = lgb_params.copy()
        params_q["objective"] = "quantile"
        # q50
        params_q50 = params_q.copy()
        params_q50["alpha"] = 0.5
        self.model_q50 = LGBMRegressor(**params_q50)
        self.model_q50.fit(X_train, y_train)

        # q90
        params_q90 = params_q.copy()
        params_q90["alpha"] = 0.9
        self.model_q90 = LGBMRegressor(**params_q90)
        self.model_q90.fit(X_train, y_train)

        self.feature_cols = cat_cols + base_cols
        self.cat_cols = cat_cols
        self.last_train_date = df["date"].max()
        self.train_metrics = {"val_mae": float(mae), "split_date": str(split_date.date())}

        elapsed = time.time() - t0
        print(f"Training finished in {elapsed:.1f}s. Val MAE: {mae:.4f}")

        return self.train_metrics

    def save(self, out_dir: str):
        Path(out_dir).mkdir(parents=True, exist_ok=True)
        joblib.dump(self.model_mean, Path(out_dir) / "model_mean.joblib")
        joblib.dump(self.model_q50, Path(out_dir) / "model_q50.joblib")
        joblib.dump(self.model_q90, Path(out_dir) / "model_q90.joblib")
        joblib.dump(self.feature_cols, Path(out_dir) / "feature_cols.joblib")
        joblib.dump(self.cat_cols, Path(out_dir) / "cat_cols.joblib")
        joblib.dump(self.train_metrics, Path(out_dir) / "train_metrics.json")
        print("Saved models to", out_dir)

    def load(self, out_dir: str):
        self.model_mean = joblib.load(Path(out_dir) / "model_mean.joblib")
        self.model_q50 = joblib.load(Path(out_dir) / "model_q50.joblib")
        self.model_q90 = joblib.load(Path(out_dir) / "model_q90.joblib")
        self.feature_cols = joblib.load(Path(out_dir) / "feature_cols.joblib")
        self.cat_cols = joblib.load(Path(out_dir) / "cat_cols.joblib")
        self.train_metrics = joblib.load(Path(out_dir) / "train_metrics.json")
        print("Loaded models from", out_dir)

    # -------------------------
    # forecast_range: autoregressive multi-SKU forecasting
    # -------------------------
    def forecast_range(self, df_all: pd.DataFrame, sku_ids: List[str], start_date: str, end_date: str,
                   use_mean_for_roll: bool = True) -> pd.DataFrame:
        """
        Faster forecast_range: only recomputes features for the requested sku_ids.
        - df_all must be the full panel (calendar+price merged) covering requested dates.
        - Returns DataFrame with ['date','id','fc_mean','fc_q50','fc_q90'] for requested SKUs.
        """
        if self.model_mean is None:
            raise RuntimeError("Model not trained. Call train(...) first.")

        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)

        # Check calendar coverage
        max_date = pd.to_datetime(df_all["date"].max())
        if end_date > max_date:
            raise ValueError(f"end_date {end_date.date()} beyond available calendar coverage {max_date.date()}.")

        # base working copy (full panel) — we'll update only rows for requested SKUs
        work = df_all.copy()
        work["date"] = pd.to_datetime(work["date"])
        work = work.sort_values(["id", "date"]).reset_index(drop=False)  # preserve original index column 'index'
        work = work.rename(columns={"index": "orig_index"})  # orig_index identifies row position in original df_all
        work["demand"] = work["demand"].astype(float)

        # ensure forecast columns exist
        work["fc_mean"] = np.nan
        work["fc_q50"] = np.nan
        work["fc_q90"] = np.nan

        # prepare the smaller subset (only requested SKUs)
        mask_sub = work["id"].isin(sku_ids)
        sub = work.loc[mask_sub].copy()   # this keeps orig_index values so we can write back easily
        # keep orig_index as index for straightforward write-back
        sub.set_index("orig_index", inplace=True)

        # cast categorical columns in sub for prediction speed
        for c in self.cat_cols:
            if c in sub.columns:
                sub[c] = sub[c].astype("category")

        # pre-create fc columns in sub
        sub["fc_mean"] = np.nan
        sub["fc_q50"] = np.nan
        sub["fc_q90"] = np.nan

        # iterate day by day only up to end_date; start from min date in work to ensure lags exist
        all_days = pd.date_range(start=work["date"].min(), end=end_date, freq="D")
        feature_cols = self.feature_cols
        cat_cols = self.cat_cols

        for day in all_days:
            # recompute features for the sub-panel (only the SKUs we care about)
            # add_panel_features uses groupby('id') and will compute lags/rollings correctly using current sub['demand']
            sub = add_panel_features(sub.reset_index(drop=False).rename(columns={"index":"orig_index"})).set_index("orig_index")
            # note: after add_panel_features we re-index by orig_index again

            # select rows for this day in sub
            day_mask_idx = sub.index[sub["date"] == day].tolist()
            if len(day_mask_idx) == 0:
                continue

            X_day = sub.loc[day_mask_idx, feature_cols].copy()

            # ensure categoricals match training
            for c in cat_cols:
                if c in X_day.columns:
                    X_day[c] = X_day[c].astype("category")

            # predict
            mean_hat = self.model_mean.predict(X_day)
            q50_hat  = self.model_q50.predict(X_day)
            q90_hat  = self.model_q90.predict(X_day)

            # write to sub (these indices are orig_index values)
            sub.loc[day_mask_idx, "fc_mean"] = mean_hat
            sub.loc[day_mask_idx, "fc_q50"]  = q50_hat
            sub.loc[day_mask_idx, "fc_q90"]  = q90_hat

            # decide what to use to fill demand for subsequent days' lags
            fill_vals = mean_hat if use_mean_for_roll else q50_hat
            sub.loc[day_mask_idx, "demand"] = fill_vals

            # also write these back into work so the "global" panel stays in sync (optional, but keeps single source)
            # we use orig_index to map rows exactly
            work.loc[day_mask_idx, ["demand", "fc_mean", "fc_q50", "fc_q90"]] = \
                sub.loc[day_mask_idx, ["demand", "fc_mean", "fc_q50", "fc_q90"]].values

        # After loop, extract requested rows for the date range
        out_mask = (work["id"].isin(sku_ids)) & (work["date"] >= start_date) & (work["date"] <= end_date)
        out = work.loc[out_mask, ["date", "id", "fc_mean", "fc_q50", "fc_q90"]].copy()
        out = out.sort_values(["id", "date"]).reset_index(drop=True)

        # clip negatives (optional safety)
        for c in ["fc_mean", "fc_q50", "fc_q90"]:
            out[c] = out[c].clip(lower=0.0)

        return out

In [ ]:
# 1) load a subset (safe for laptop)
df = load_m5_long("../data/raw/m5", max_items=100)   # reduce max_items if low RAM

# 2) train
svc = GlobalForecastModel()
metrics = svc.train(df, val_days=28)
print(metrics)

# 3) save
svc.save("../models/global_forecast_v1")


Training until validation scores don't improve for 100 rounds
[100]	valid_0's l2: 6.69318
[200]	valid_0's l2: 6.68694
Early stopping, best iteration is:
[134]	valid_0's l2: 6.65348
Training finished in 75.4s. Val MAE: 1.1289
{'val_mae': 1.128860167111154, 'split_date': '2016-03-27'}
Saved models to models/global_forecast_v1


In [35]:
import pandas as pd
import numpy as np

def extend_panel_with_future(df_all: pd.DataFrame, m5_dir: str, end_date: str) -> pd.DataFrame:
    """
    Extends df_all to include future dates up to end_date for all SKUs already present in df_all.
    Pulls future calendar rows from calendar.csv and prices from sell_prices.csv.

    Assumes df_all already has these columns:
      id, item_id, store_id, state_id, dept_id, cat_id, date, wm_yr_wk, price, and calendar fields.

    Returns: extended dataframe including new future rows (demand is NaN for new rows).
    """
    end_date = pd.to_datetime(end_date)
    df_all = df_all.copy()
    df_all["date"] = pd.to_datetime(df_all["date"])

    current_max = df_all["date"].max()
    if end_date <= current_max:
        return df_all

    # Load calendar + prices
    cal = pd.read_csv(f"{m5_dir}/calendar.csv")
    cal["date"] = pd.to_datetime(cal["date"])
    prices = pd.read_csv(f"{m5_dir}/sell_prices.csv").rename(columns={"sell_price": "price"})

    # Dates we need to add
    future_cal = cal[(cal["date"] > current_max) & (cal["date"] <= end_date)].copy()
    if future_cal.empty:
        raise ValueError(f"No calendar rows available after {current_max.date()} up to {end_date.date()}")

    # Unique SKU identities from current panel
    sku_meta = df_all[["id","item_id","dept_id","cat_id","store_id","state_id"]].drop_duplicates()

    # Create cartesian product: (SKUs) x (future dates)
    future_rows = sku_meta.merge(future_cal, how="cross")

    # Merge prices on (store_id, item_id, wm_yr_wk)
    future_rows = future_rows.merge(prices, on=["store_id","item_id","wm_yr_wk"], how="left")

    # Demand is unknown in future rows
    future_rows["demand"] = np.nan

    # Align columns to df_all
    missing_cols = [c for c in df_all.columns if c not in future_rows.columns]
    for c in missing_cols:
        future_rows[c] = np.nan

    # Keep same column order as df_all
    future_rows = future_rows[df_all.columns]

    out = pd.concat([df_all, future_rows], ignore_index=True)
    out = out.sort_values(["id","date"]).reset_index(drop=True)
    return out

In [ ]:
df_ext = extend_panel_with_future(df, m5_dir="../data/raw/m5", end_date="2016-04-28")

sku_list = df_ext["id"].drop_duplicates().head(10).tolist()
fc = svc.forecast_range(df_ext, sku_list, start_date="2016-04-01", end_date="2016-04-28")
fc.head()

,date,id,fc_mean,fc_q50,fc_q90
0,2016-04-01,HOBBIES_1_001_CA_1_validation,0.467934,0.006674,1.976701
1,2016-04-02,HOBBIES_1_001_CA_1_validation,0.522983,0.104836,1.922672
2,2016-04-03,HOBBIES_1_001_CA_1_validation,0.522983,0.085417,1.909235
3,2016-04-04,HOBBIES_1_001_CA_1_validation,0.416908,0.024746,1.812157
4,2016-04-05,HOBBIES_1_001_CA_1_validation,0.416908,0.006020,1.699342


In [40]:
df_ext.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,date,wm_yr_wk,...,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,price
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0.0,2011-01-29,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
1,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_2,0.0,2011-01-30,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
2,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_3,0.0,2011-01-31,11101,...,1,2011,NaN,NaN,NaN,NaN,0,0,0,NaN
3,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_4,0.0,2011-02-01,11101,...,2,2011,NaN,NaN,NaN,NaN,1,1,0,NaN
4,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_5,0.0,2011-02-02,11101,...,2,2011,NaN,NaN,NaN,NaN,1,0,1,NaN


In [9]:
import pandas as pd
import numpy as np
from pathlib import Path

M5_DIR = Path("../data/raw/m5")   # change if needed

def build_df_base(store_ids=("CA_1",), max_ids=3000):
    """
    Builds the base dataframe used by all agents.
    """
    sales = pd.read_csv(M5_DIR / "sales_train_validation.csv")
    calendar = pd.read_csv(M5_DIR / "calendar.csv")
    prices = pd.read_csv(M5_DIR / "sell_prices.csv")

    # Filter to selected stores (regions)
    sales = sales[sales["store_id"].isin(store_ids)].copy()

    # Limit number of SKU-store series (for stability)
    keep_ids = sales["id"].drop_duplicates().head(max_ids)
    sales = sales[sales["id"].isin(keep_ids)].copy()

    # Convert wide -> long
    d_cols = [c for c in sales.columns if c.startswith("d_")]
    id_cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]

    df = sales[id_cols + d_cols].melt(
        id_vars=id_cols,
        var_name="d",
        value_name="demand"
    )

    # Merge calendar (date + week)
    calendar["date"] = pd.to_datetime(calendar["date"])
    df = df.merge(
        calendar[["d", "date", "wm_yr_wk"]],
        on="d",
        how="left"
    )

    # Merge prices
    prices = prices.rename(columns={"sell_price": "price"})
    df = df.merge(
        prices[["store_id", "item_id", "wm_yr_wk", "price"]],
        on=["store_id", "item_id", "wm_yr_wk"],
        how="left"
    )

    # Clean types
    df["date"] = pd.to_datetime(df["date"])
    df["demand"] = df["demand"].astype(np.float32)
    df["price"] = df["price"].astype(np.float32)

    # Sort properly
    df = df.sort_values(["id", "date"]).reset_index(drop=True)

    return df

In [2]:
import sys
from pathlib import Path
project_root = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
sys.path.insert(0, str(project_root))
from backend.forecast_agent import ForecastAgent


In [ ]:
from google import genai
import os

# Set env var first in Colab:
# os.environ["GEMINI_API_KEY"] = "YOUR_KEY"

client = genai.Client(api_key="")

agent = ForecastAgent(
    model_dir="../models",
    m5_dir="../data/raw/m5",
    llm_client=client   # Gemini
)

out = agent.forecast(
    item_ids=["FOODS_3_090", "HOBBIES_1_001"],
    start_date="2016-03-28",
    end_date="2016-04-24"
)

print(out["llm_summary"])

Based on our latest projections for the four-week period starting March 28th, we expect steady sales activity at the CA_1 store. Total demand across these items is estimated at 1,450 units. The majority of this volume is driven by a high-demand food item, which we expect customers to buy at a rate of about 51 units every day. In contrast, the hobby item moves much more slowly, with an average of only one sale per day. These figures suggest that while the food item requires constant attention, the hobby item will have very low turnover. Overall, our goal is to keep these products available for customers without keeping too much extra stock in the back.

**Practical Planning Suggestion:**
To maximize efficiency, schedule daily or every-other-day restocking for the high-volume food item to ensure it never runs out, while ordering the entire month's supply of the hobby item in a single small shipment to save on delivery efforts.


In [11]:
out.keys()

dict_keys(['start_date', 'end_date', 'results', 'llm_summary', 'model_metrics'])

In [12]:
out['results']

[{'item_id': 'FOODS_3_090',
  'store_id': 'CA_1',
  'series_id': 'FOODS_3_090_CA_1_validation',
  'daily_forecast':            date  forecast
  1885 2016-03-28      25.0
  1886 2016-03-29      47.0
  1887 2016-03-30      23.0
  1888 2016-03-31      21.0
  1889 2016-04-01      72.0
  1890 2016-04-02      58.0
  1891 2016-04-03      54.0
  1892 2016-04-04      42.0
  1893 2016-04-05      29.0
  1894 2016-04-06      53.0
  1895 2016-04-07      39.0
  1896 2016-04-08      42.0
  1897 2016-04-09      83.0
  1898 2016-04-10      83.0
  1899 2016-04-11      48.0
  1900 2016-04-12      26.0
  1901 2016-04-13      41.0
  1902 2016-04-14      44.0
  1903 2016-04-15      47.0
  1904 2016-04-16      82.0
  1905 2016-04-17      83.0
  1906 2016-04-18      30.0
  1907 2016-04-19      45.0
  1908 2016-04-20      29.0
  1909 2016-04-21      53.0
  1910 2016-04-22      87.0
  1911 2016-04-23      95.0
  1912 2016-04-24      42.0,
  'total_units': 1423.0},
 {'item_id': 'HOBBIES_1_001',
  'store_id': 'CA

In [4]:
from backend.simulation_agent import SimulationAgent

In [5]:
forecast_df = out['results'][0]["daily_forecast"]

In [6]:
sim_agent = SimulationAgent(
    n_simulations=500,
    demand_cv=0.25,
    lead_time_days=2
)

sim_result = sim_agent.simulate(
    forecast_df=forecast_df,
    item_id="FOODS_3_090",
    store_id="CA_1",
    s=80,
    Q=120
)

sim_result

{'item_id': 'FOODS_3_090',
 'store_id': 'CA_1',
 'policy': {'s': 80, 'Q': 120},
 'lead_time_days': 2,
 'initial_inventory': 80,
 'results': {'expected_fill_rate': 0.8617199243804855,
  'stockout_probability': 1.0,
  'avg_stockout_days': 5.592,
  'expected_lost_units': 198.29348497883825,
  'avg_inventory': 48.05153724327553,
  'avg_orders': 10.708},
 'scenario_summary': {'mean_total_demand': 1423.6870711196088,
  'p95_total_demand': 1539.8132012016497}}

In [7]:
selected_products = ["FOODS_3_090", "HOBBIES_1_001"]

In [28]:
results = []
for i in range(len(selected_products)):
    sim = sim_agent.simulate(
        forecast_df=out['results'][i]["daily_forecast"],
        item_id=selected_products[i],
        store_id="CA_1",
        s=100,
        Q=350
    )
    results.append(sim)

In [29]:
results

[{'item_id': 'FOODS_3_090',
  'store_id': 'CA_1',
  'policy': {'s': 100, 'Q': 350},
  'lead_time_days': 2,
  'initial_inventory': 201,
  'results': {'expected_fill_rate': 0.9677501310317528,
   'stockout_probability': 0.736,
   'avg_stockout_days': 1.438,
   'expected_lost_units': 45.19710978373548,
   'avg_inventory': 161.8198035413653},
  'scenario_summary': {'mean_total_demand': 1423.2762855293638,
   'p95_total_demand': 1544.187539081274}},
 {'item_id': 'HOBBIES_1_001',
  'store_id': 'CA_1',
  'policy': {'s': 100, 'Q': 350},
  'lead_time_days': 2,
  'initial_inventory': 101,
  'results': {'expected_fill_rate': 1.0,
   'stockout_probability': 0.0,
   'avg_stockout_days': 0.0,
   'expected_lost_units': 0.0,
   'avg_inventory': 391.55832896136764},
  'scenario_summary': {'mean_total_demand': 32.69605548435732,
   'p95_total_demand': 39.61397839618095}}]

In [10]:
df_base = build_df_base(
    store_ids=("CA_1", "TX_1", "WI_1"),  # all regions
    max_ids=3000                         # safe size
)

print(df_base.shape)

(5739000, 11)


In [11]:
avg_price = (
    df_base[
        (df_base["item_id"] == "FOODS_3_090") &
        (df_base["store_id"] == "CA_1")
    ]["price"].mean()
)

In [12]:
from backend.optimization_agent import OptimizationAgent

In [17]:
sim_agent = SimulationAgent(
    n_simulations=500,
    demand_cv=0.25,
    lead_time_days=2
)

sim_result = sim_agent.simulate(
    forecast_df=out['results'][0]["daily_forecast"],
    item_id="FOODS_3_090",
    store_id="CA_1",
    s=80,
    Q=120
)

sim_result

{'item_id': 'FOODS_3_090',
 'store_id': 'CA_1',
 'policy': {'s': 80, 'Q': 120},
 'lead_time_days': 2,
 'initial_inventory': 80,
 'results': {'expected_fill_rate': 0.8617199243804855,
  'stockout_probability': 1.0,
  'avg_stockout_days': 5.592,
  'expected_lost_units': 198.29348497883825,
  'avg_inventory': 48.05153724327553,
  'avg_orders': 10.708},
 'scenario_summary': {'mean_total_demand': 1423.6870711196088,
  'p95_total_demand': 1539.8132012016497}}

In [20]:
opt_agent = OptimizationAgent(
    simulation_agent=sim_agent,
    holding_cost_rate=0.25,        # 25% annually
    order_cost=20.0,               # $20 per order
    stockout_cost_multiplier=3.0,  # lost sales penalty
    target_fill_rate=0.95
)

# Candidate policies to try
s_candidates = range(50, 201, 25)
Q_candidates = range(100, 401, 50)

opt_result = opt_agent.optimize(
    forecast_df=out['results'][0]["daily_forecast"],
    item_id="FOODS_3_090",
    store_id="CA_1",
    avg_price=avg_price,
    s_candidates=s_candidates,
    Q_candidates=Q_candidates
)

In [21]:
opt_result["best_policy"]

{'item_id': 'FOODS_3_090',
 'store_id': 'CA_1',
 's': 200,
 'Q': 400,
 'fill_rate': 0.9991917685612025,
 'total_cost': 92.85790252685547,
 'holding_cost': 7.7258453369140625,
 'ordering_cost': 80.16,
 'stockout_cost': 4.972055435180664}